# Application of Fuzzy logic and Genetic Algorithms in Stock Trading

This notebook explores the application of genetic algorithms in optimizing the fuzzy rules as applied in technical analysis of stocks.